In [1]:
import cv2
import os
from keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
!pip install pygame

In [2]:
# Initialize pygame mixer for sound alerts
mixer.init()
alarm_sound = mixer.Sound("alarm.wav")

# Load Haar cascade classifiers for face and eyes
face_cascade = cv2.CascadeClassifier("haarcascade_frontalface_alt.xml")
left_eye_cascade = cv2.CascadeClassifier("haarcascade_lefteye_2splits.xml")
right_eye_cascade = cv2.CascadeClassifier("haarcascade_righteye_2splits.xml")

# Load trained drowsiness detection model
model = load_model('drowsiness_detector.h5')

# Initialize webcam
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_SIMPLEX
score = 0
thickness = 2
alert_threshold = 10  # Reduced threshold for quicker alert response

while True:
    ret, frame = cap.read()
    if not ret:
        break
    height, width = frame.shape[:2]
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # Detect faces, left eye, and right eye
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(25, 25))
    left_eyes = left_eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))
    right_eyes = right_eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(20, 20))

    lpred, rpred = [1], [1]  # Default to eyes open
    
    for (x, y, w, h) in right_eyes:
        r_eye = gray[y:y+h, x:x+w]
        r_eye = cv2.resize(r_eye, (24, 24), interpolation=cv2.INTER_AREA)
        r_eye = r_eye / 255.0
        r_eye = np.expand_dims(r_eye.reshape(24, 24, -1), axis=0)
        rpred = np.argmax(model.predict(r_eye), axis=1)
        break
    
    for (x, y, w, h) in left_eyes:
        l_eye = gray[y:y+h, x:x+w]
        l_eye = cv2.resize(l_eye, (24, 24), interpolation=cv2.INTER_AREA)
        l_eye = l_eye / 255.0
        l_eye = np.expand_dims(l_eye.reshape(24, 24, -1), axis=0)
        lpred = np.argmax(model.predict(l_eye), axis=1)
        break

    # Update score based on eye status
    if rpred[0] == 0 and lpred[0] == 0:
        score += 1
        cv2.putText(frame, "Eyes Closed", (10, height - 20), font, 0.7, (0, 0, 255), 2)
    else:
        score = max(0, score - 1)
        cv2.putText(frame, "Eyes Open", (10, height - 20), font, 0.7, (0, 255, 0), 2)
    
    cv2.putText(frame, f"Drowsiness Score: {score}", (width - 220, height - 20), font, 0.7, (255, 255, 255), 2)

    # Trigger alarm if drowsiness score is too high
    if score >= alert_threshold:
        cv2.imwrite(os.path.join(os.getcwd(), "alert.png"), frame)
        try:
            alarm_sound.play()
        except Exception as e:
            print("Error playing sound:", e)
        
        thickness = min(thickness + 2, 16)
        cv2.rectangle(frame, (0, 0), (width, height), (0, 0, 255), thickness)
    else:
        thickness = max(thickness - 2, 2)
    
    cv2.imshow("Drowsiness Detection", frame)
    
    if cv2.waitKey(1) & 0xFF == ord("q"):
        print("Exiting Drowsiness Detection...")
        break

cap.release()
cv2.destroyAllWindows()

C:\Users\anami\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\anami\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━